# [2.X코드반영] Lab 10-4: Batch Normalization

### Import libraries

In [21]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.4.1


### Checkpoint function

In [22]:
def load(model, checkpoint_dir):
  print(' [*] Reading checkpoints...')

  ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
  if ckpt:
    ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
    checkpoint = tf.train.Checkpoint(dnn=model)
    checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
    counter = int(ckpt_name.split('-')[1])
    print(' [*] Success to read {}'.format(ckpt_name))
    return True, counter
  else:
    print(' [*] Failed to find a checkpoint')
    return False, 0

def check_folder(dir):
  if not os.path.exists(dir):
    os.makedirs(dir)
  return dir

### Data load & preprocessing function

In [23]:
def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
  train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
  test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

  train_data, test_data = normalize(train_data, test_data)

  train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
  test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

  return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0

  return train_data, test_data

### Performance function

In [24]:
def loss_fn(model, images, labels):
  # training=True : dropout 사용
  logits = model(images, training=True)
  loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
  return loss

def accuracy_fn(model, images, labels):
  # training=False : dropout 미사용
  logits = model(images, training=False)
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
  return accuracy

def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.trainable_variables)

### Model function

In [25]:
def flatten():
  return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
  return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm():
  return tf.keras.layers.BatchNormalization()

### Create model (class version)

In [26]:
def creat_model_function(label_dim):
  def __init__(self, label_dim):
    super(creat_model_function, self).__init__()
    weight_init = tf.keras.initializers.glorot_uniform()
    self.model = tf.keras.Sequential()
    self. model.add(flatten())

    for i in range(4):
      self.model.add(dense(512, weight_init))
      self.model.add(batch_norm())
      self.model.add(relu())

    self.model.add(dense(label_dim, weight_init))
    
  def call(self, x, training=None, mask=None):
    x = self.model(x)
    return x

### Create model (function version)

In [27]:
def creat_model_function(label_dim):
  weight_init = tf.keras.initializers.glorot_uniform()
  model = tf.keras.Sequential()
  model.add(flatten())

  for i in range(4):
    model.add(dense(512, weight_init))
    model.add(batch_norm())
    model.add(relu())

  model.add(dense(label_dim, weight_init))

  return model

### Define data & hyper parameter


In [28]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=batch_size).\
  batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=len(test_x)).\
  batch(len(test_x))

### Define model & optimizer & writer

In [29]:
# model
network = creat_model_function(label_dim)

# training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batch_norm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [30]:
if train_flag:
  checkpoint = tf.train.Checkpoint(dnn=network)

  # create writer for tensorboard
  summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
  start_time = time()

  # restore checkpoint if it exists
  could_load, checkpoint_counter = load(network, checkpoint_dir)

  if could_load:
    start_epoch = (int)(checkpoint_counter/training_iterations)
    counter = checkpoint_counter
    print(' [*] Load success')
  else:
    start_epoch = 0
    start_iteration = 0
    counter = 0
    print(' [!] Load failed')

    # train phase
    with summary_writer.as_default(): # for tensorboard
      for epoch in range(start_epoch, training_epochs):
        for idx, (train_input, train_label) in enumerate(train_dataset):
          grads = grad(network, train_input, train_label)
          optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))
          train_loss = loss_fn(network, train_input, train_label)
          train_accuracy = accuracy_fn(network, train_input, train_label)

          for test_input, test_label in test_dataset:
            test_accuracy = accuracy_fn(network, test_input, test_label)
          
          tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
          tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
          tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

          print(
              'Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f'\
              %(epoch, idx, training_iterations, time()-start_time, train_loss, train_accuracy, test_accuracy)
          )
          counter += 1
      checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(counter))

# test phase
else:
  _, _ = load(network, checkpoint_dir)
  for test_input, test_label in test_dataset:
    test_accuracy = accuracy_fn(network, test_input, test_label)

  print('test_accuracy: %.4f' %(test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed
Epoch: [ 0] [    0/  468] time: 1.0673, train_loss: 0.30629557, train_accuracy: 0.3906, test_accuracy: 0.2492
Epoch: [ 0] [    1/  468] time: 1.7036, train_loss: 0.27387089, train_accuracy: 0.3594, test_accuracy: 0.2971
Epoch: [ 0] [    2/  468] time: 2.3404, train_loss: 0.34343648, train_accuracy: 0.3594, test_accuracy: 0.3879
Epoch: [ 0] [    3/  468] time: 2.9737, train_loss: 0.30070961, train_accuracy: 0.5469, test_accuracy: 0.5158
Epoch: [ 0] [    4/  468] time: 3.6131, train_loss: 0.23420238, train_accuracy: 0.7109, test_accuracy: 0.6265
Epoch: [ 0] [    5/  468] time: 4.2479, train_loss: 0.30731454, train_accuracy: 0.7344, test_accuracy: 0.7010
Epoch: [ 0] [    6/  468] time: 4.8791, train_loss: 0.30954373, train_accuracy: 0.7656, test_accuracy: 0.7411
Epoch: [ 0] [    7/  468] time: 5.5195, train_loss: 0.23436454, train_accuracy: 0.7812, test_accuracy: 0.7690
Epoch: [ 0] [    8/  468] time: 6.1520, tr

Test accuracy : 97% 대